In [1]:
!nvidia-smi

Mon Apr  7 17:25:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A4000               On  |   00000000:01:00.0 Off |                  Off |
| 41%   38C    P8             15W /  140W |       2MiB /  16376MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
# Uninstall potentially conflicting packages for a clean slate
!pip uninstall -y transformers torch torchvision torchaudio tensorflow tf-keras datasets accelerate

# Install PyTorch with CUDA support (adjust cuXXX if needed for your environment)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Install the LATEST stable versions of transformers and other required libraries
!pip install transformers datasets wandb huggingface_hub accelerate

# Verify installation
!pip list | grep -E 'torch|transformers|datasets|accelerate'
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version used by torch: {torch.version.cuda}")
import transformers
print(f"Transformers version: {transformers.__version__}")
import accelerate
print(f"Accelerate version: {accelerate.__version__}")

Found existing installation: transformers 4.51.0
Uninstalling transformers-4.51.0:
  Successfully uninstalled transformers-4.51.0
Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tf_keras 2.19.0
Uninstalling tf_keras-2.19.0:
  Successfully uninstalled tf_keras-2.19.0
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 MB 15.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 88.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [10]:
# Import necessary libraries
import os
import wandb
from datasets import load_dataset
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
import torch

torch.manual_seed(9001)
import random
random.seed(9001)

# Initialize wandb (make sure you are logged in with `wandb login`)
wandb.init(project="gpt2-large-countdown", name="gpt2-large-countdown-run-1", reinit=True)

RuntimeError: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [9]:
# Load the datasets
data_files = {
    "train": "train.jsonl",
    "validation": "validation.jsonl",
    "test_1e3": "test_ood_1e3.jsonl",
    "test_1e4": "test_ood_1e4.jsonl",
    "test_1e5": "test_ood_1e5.jsonl"
}
dataset = load_dataset("json", data_files=data_files)

# Combine the fields into a single prompt.
def preprocess(example):
    """ Here we combine numbers, target, and the chain-of-thought (CoT) into one string. """
    cot_text = " ".join(example["cot"])
    text = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {example['numbers']}, create an equation that equals {example['target']}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>{cot_text}</think>
<answer>{example['target']}</answer>"""
    return {"text": text}

# Map the preprocessing function over the dataset splits
dataset = dataset.map(preprocess, batched=False)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test_1e3 split: 0 examples [00:00, ? examples/s]

Generating test_1e4 split: 0 examples [00:00, ? examples/s]

Generating test_1e5 split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [10]:
# Load the GPT-2 small tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
# GPT-2 doesn't have a pad token by default; we set it equal to the EOS token.
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["numbers", "target", "cot", "text"])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
# Data collator for causal language modeling (no MLM masking needed)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [33]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-large-countdown",
    eval_strategy="steps", # Evaluate at set steps instead of at the end of each epoch
    eval_steps=500, # Evaluate every 500 steps
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    push_to_hub=True, # Automatically push checkpoints and final model to the Hub
    hub_model_id="giordanorogers/gpt2-large-countdown", # Replace with your repo ID
    load_best_model_at_end=True, # Automatically load the best model based on eval_loss
    metric_for_best_model="eval_loss",
    report_to=["wandb"]
)

# Load the GPT-2 model from Hugging Face Hub
model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2-large", from_tf=True)
# Set the pad_token_id to avoid warnings during training
model.config.pad_token_id = tokenizer.pad_token_id

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
)

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

/venv/main/lib/python3.10/site-packages/huggingface_hub/file_download.py:752: UserWarning: Not enough free disk space to download the file. The expected file size is: 3247.16 MB. The target location /root/.cache/huggingface/hub/models--openai-community--gpt2-large/blobs only has 0.74 MB free disk space.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

/venv/main/lib/python3.10/site-packages/huggingface_hub/file_download.py:752: UserWarning: Not enough free disk space to download the file. The expected file size is: 3247.20 MB. The target location /root/.cache/huggingface/hub/models--openai-community--gpt2-large/blobs only has 0.74 MB free disk space.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

OSError: openai-community/gpt2-large does not appear to have a file named pytorch_model.bin but there is a file for TensorFlow weights. Use `from_tf=True` to load this model from those weights.

In [ ]:
trainer.train()

trainer.push_to_hub()

In [ ]:
# Evaluate the model on the OOD test set
eval_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
print("OOD Test Evaluation:", eval_results)

In [ ]:
from transformers import pipeline, GPT2Tokenizer, GPT2LMHeadModel

# Load the trained model and tokenizer from the local directory (or replace with your Hub repo ID)
model = GPT2LMHeadModel.from_pretrained("giordanorogers/gpt2-large-countdown")  # or "giordanorogers/gpt2-small-countdown" if pushed
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure pad_token is set

# Create a text generation pipeline
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define a prompt to test your model
numbers = [16, 44, 57, 62, 71, 75]

target = 1557

prompt = f"""A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
User: Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>.
Assistant: Let me solve this step by step.
<think>"""

# Generate output
output = generator(prompt, max_length=300, truncation=True, num_return_sequences=1)

# Print the generated outputa
print(output[0]["generated_text"])